In [1]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
np.random.seed(1337)  # for reproducibility
from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Embedding
from keras.layers import LSTM, SimpleRNN, GRU
from keras.utils.np_utils import to_categorical
from sklearn.metrics import (precision_score, recall_score,f1_score, accuracy_score,mean_squared_error,mean_absolute_error)
from sklearn import metrics
from sklearn.preprocessing import Normalizer
import h5py
from keras import callbacks
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger
from keras.models import Sequential
from keras.layers import Dense
import tensorflow as tf
import keras
from keras.utils.np_utils import to_categorical
from keras import models
from keras import layers
import numpy
import sys
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, MinMaxScaler
import math
from sklearn.externals import joblib
import joblib
from sklearn.preprocessing import StandardScaler
from keras.layers import BatchNormalization
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score

# load  preprocessed dataset
data=pd.read_csv("train.csv")
data_new=data.values

X=data_new[:,:-1] 
Y=data_new[:,-1]

#train-test split
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split( X, Y, test_size=0.30)

print(x_train.shape)
print(x_test.shape)

import joblib
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
x_train=scaler.fit_transform(x_train)
joblib.dump(scaler, 'hyperopt_ss.pkl')
x_test=scaler.transform(x_test)



Using TensorFlow backend.
E:\ProgramData\Anaconda3\envs\tf_gpu\lib\site-packages\sklearn\externals\joblib\__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


(732980, 20)
(314135, 20)


In [2]:
## required for efficient GPU use
import tensorflow as tf
from keras.backend import tensorflow_backend
config = tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True))
session = tf.Session(config=config)
tensorflow_backend.set_session(session)

In [3]:
import hyperopt
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
space = {'learning_rate': hp.loguniform('learning_rate',np.log(0.0001),np.log(0.2)),'batch_size' : hp.uniform('batch_size', 8,256),'epochs' : 100}

In [4]:
def hyperparameters(space):   
    from keras.preprocessing import sequence
    from keras.utils import np_utils
    from keras.models import Sequential
    from keras.layers import Dense, Dropout, Activation, Embedding
    from keras.layers import LSTM, SimpleRNN, GRU
    from keras.utils.np_utils import to_categorical
    from sklearn.metrics import (precision_score, recall_score,f1_score, accuracy_score,mean_squared_error,mean_absolute_error)
    from sklearn import metrics
    from sklearn.preprocessing import Normalizer
    import h5py
    from keras import callbacks
    from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger
    from keras.models import Sequential
    import tensorflow as tf
    import keras
    from keras.utils.np_utils import to_categorical
    from keras import models
    from keras import layers


    print ('Params testing: ', space)
    model= Sequential()
    model.add(Dense(1024,input_dim=20,activation='relu'))  

    model.add(Dropout(0.1))

    model.add(Dense(512,activation='relu'))  

    model.add(Dropout(0.1))

    model.add(Dense(256,activation='relu'))  

    model.add(Dropout(0.05))

    model.add(Dense(128,activation='relu'))  

    model.add(Dropout(0.05))

    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    
    optimizer = keras.optimizers.Adam(lr=float(space['learning_rate']))
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy','mse', 'mae'])
    model.summary()  

    model.fit(x_train,y_train,epochs= int(space['epochs']),validation_data=(x_test,y_test),batch_size=int(space['batch_size']),verbose=0)

    pred_auc =model.predict_proba(x_test, batch_size= int(space['batch_size']), verbose = 0)
    acc = roc_auc_score(y_test, pred_auc)
    print('AUC:', acc)
    sys.stdout.flush() 
    return {'loss': acc, 'status': STATUS_OK}

In [5]:
from hyperopt import rand
trials = Trials()
best = fmin(fn=hyperparameters,
            space=space,
            algo=tpe.suggest,
            max_evals=5,
            trials=trials)

Params testing:                                                                                                        
{'batch_size': 149.3600110574449, 'epochs': 100, 'learning_rate': 0.004854043303943122}                                
  0%|                                                                              | 0/5 [00:00<?, ?it/s, best loss: ?]


W0305 13:34:34.121934  6916 deprecation_wrapper.py:119] From E:\ProgramData\Anaconda3\envs\tf_gpu\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.


W0305 13:34:34.122937  6916 deprecation_wrapper.py:119] From E:\ProgramData\Anaconda3\envs\tf_gpu\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.


W0305 13:34:34.126934  6916 deprecation_wrapper.py:119] From E:\ProgramData\Anaconda3\envs\tf_gpu\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.


W0305 13:34:34.149930  6916 deprecation_wrapper.py:119] From E:\ProgramData\Anaconda3\envs\tf_gpu\lib\site-packages\keras\backend\tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default inst

_________________________________________________________________                                                      
Layer (type)                 Output Shape              Param #                                                         
dense_1 (Dense)              (None, 1024)              21504                                                           
_________________________________________________________________                                                      
dropout_1 (Dropout)          (None, 1024)              0                                                               
_________________________________________________________________                                                      
dense_2 (Dense)              (None, 512)               524800                                                          
_________________________________________________________________                                                      
dropout_2 (Dropout)          (None, 512)

_________________________________________________________________                                                      
dropout_19 (Dropout)         (None, 256)               0                                                               
_________________________________________________________________                                                      
dense_24 (Dense)             (None, 128)               32896                                                           
_________________________________________________________________                                                      
dropout_20 (Dropout)         (None, 128)               0                                                               
_________________________________________________________________                                                      
dense_25 (Dense)             (None, 1)                 129                                                             
________________________________________

In [7]:
##References
#https://towardsdatascience.com/hyperparameter-optimization-in-python-part-2-hyperopt-5f661db91324
#https://campus.datacamp.com/courses/hyperparameter-tuning-in-python/informed-search?ex=6